# WABAD integration

In [1]:
import pandas as pd
import os 
import json
import csv
from tqdm import tqdm
from pydub import AudioSegment
import numpy as np
import librosa
import soundfile as sf

In [2]:
from pathlib import Path

all_species = Path("utils/custom_species_list.txt").read_text(encoding="utf-8").splitlines()
all_species = [specie.split("_")[0] for specie in all_species]
all_species[:5]

['Wind',
 'Regulus ignicapilla',
 'Sylvia atricapilla',
 'Fringilla coelebs',
 'Troglodytes troglodytes']

In [3]:
# import csv with pandas
df_orig = pd.read_csv('utils/Pooled annotations.csv', delimiter=';', encoding='ISO-8859-1')
df_orig.head()

,Recording site,Recording,Species,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Country,Continent,Biome
0,ARD,ARD_20211027_072000.wav,Vireo chivi,"0,191831617","0,785890816",2000,"4351,4",Argentina,South America,Tropical and Subtropical Moist Broadleaf Forest
1,ARD,ARD_20211027_072000.wav,Dysithamnus mentalis,"0,0371287","1,664603382","1378,4","2243,2",Argentina,South America,Tropical and Subtropical Moist Broadleaf Forest
2,ARD,ARD_20211027_072000.wav,Euphonia chlorotica,"2,827969315","3,669553181","2432,4","4729,7",Argentina,South America,Tropical and Subtropical Moist Broadleaf Forest
3,ARD,ARD_20211027_072000.wav,Vireo chivi,"4,622523147","5,259899164","1918,9","4405,4",Argentina,South America,Tropical and Subtropical Moist Broadleaf Forest
4,ARD,ARD_20211027_072000.wav,Thamnophilus caerulescens,"5,349498323","7,057418522","864,9","2216,2",Argentina,South America,Tropical and Subtropical Moist Broadleaf Forest


In [4]:
len(df_orig["Recording site"].unique())

70

In [5]:
# group by species, select only the species in the all_species list
df = df_orig[df_orig['Species'].isin(all_species)]
df.head()

,Recording site,Recording,Species,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Country,Continent,Biome
3671,BAM,BAM_20151130_091318.wav,Phylloscopus collybita,"51,17683904","51,46307236","2292,8",5171,Cameroon,Africa,Tropical and Subtropical Moist Broadleaf Forest
3672,BAM,BAM_20151130_091318.wav,Phylloscopus collybita,"58,9119536","59,13003612","2650,6","5008,4",Cameroon,Africa,Tropical and Subtropical Moist Broadleaf Forest
3816,BAM,BAM_20151207_071718.wav,Phylloscopus trochilus,"18,53027398","22,16482932","1641,6",6809,Cameroon,Africa,Tropical and Subtropical Moist Broadleaf Forest
4459,BIAL,BIAL_20210420_051500.wav,Phylloscopus collybita,"0,078707352","3,899591521","3083,004","7541,502",Poland,Europe,Temperate Broadleaf and Mixed Forest
4460,BIAL,BIAL_20210420_051500.wav,Phylloscopus collybita,"7,162369014","10,8902354","2928,854","7102,767",Poland,Europe,Temperate Broadleaf and Mixed Forest


In [6]:
# numeri VERIFICATI con i file originali
df.groupby("Species").size().sort_values(ascending=False)

Species
Fringilla coelebs          3324
Turdus merula              2308
Erithacus rubecula         1395
Sylvia atricapilla         1012
Turdus philomelos           833
Phylloscopus collybita      819
Periparus ater              768
Troglodytes troglodytes     530
Regulus regulus             437
Phylloscopus trochilus      418
Regulus ignicapilla         286
Dendrocopos major           242
Turdus viscivorus           229
Lophophanes cristatus       132
Muscicapa striata           117
Spinus spinus               102
Certhia familiaris           96
Prunella modularis           75
Dryocopus martius            32
Loxia curvirostra            25
Poecile palustris            12
Pyrrhula pyrrhula             8
dtype: int64

In [7]:
df.groupby("Recording site").size().sort_values(ascending=False)

Recording site
BIAL      1628
HAG       1452
SLOB      1244
SITH      1108
NAV       1000
DYOM       886
KAR        859
CAT        787
SCHF       680
PINA       675
GLEN       628
CLH        592
SCHG       418
OLIV       255
HAR        237
VIL        222
MOPU       198
COU         79
HONDO       76
BOLIN       61
ROTOK       39
FEU         37
PITI        18
POZO        12
BAM          3
OESF         3
MONTEB       2
CB           1
dtype: int64

1. verifica quante annotazioni richiede ogni specie
2. ricreare audio_info e category_info per le nuove cartelle
3. refill cartelle con i nuovi segmenti

## 1. train and test count

In [36]:
training_folder = "E:/Giacomo/Tovanella-20241110T120546Z-001/test_set_augm/train"
wabad_training_folder = "E:/Giacomo/Tovanella-20241110T120546Z-001/dataset_wabad_boosted/train"
test_folder = "E:/Giacomo/Tovanella-20241110T120546Z-001/test_set_augm/test"

# list folder in training_folder
species_folders = os.listdir(wabad_training_folder)
# list all elements in each folder in folders
species_count = { folder: {
    # "train_count": len(os.listdir(os.path.join(training_folder, folder))),
    "WABAD_train_count": len(os.listdir(os.path.join(wabad_training_folder, folder))),
    "test_count": len(os.listdir(os.path.join(test_folder, folder)))
    } for folder in species_folders }

df = pd.DataFrame(species_count).T

In [25]:
df.sort_values(["WABAD_train_count"], ascending=False)

,WABAD_train_count,test_count
Fringilla coelebs_Common Chaffinch,3388,33
Sylvia atricapilla_Eurasian Blackcap,2730,225
Turdus philomelos_Song Thrush,2387,28
Troglodytes troglodytes_Eurasian Wren,1713,187
Periparus ater_Coal Tit,1448,41
Regulus ignicapilla_Common Firecrest,1334,26
Erithacus rubecula_European Robin,1282,23
Turdus merula_Eurasian Blackbird,1202,4
Phylloscopus collybita_Common Chiffchaff,1069,1030
Phylloscopus trochilus_Willow Warbler,683,5


In [37]:
# write a function that randomly move files from a folder to another until you have n_files
def move_files_randomly(source_folder, destination_folder, n_files):
    files = os.listdir(source_folder)
    files = np.random.choice(files, n_files, replace=False)
    for file in files:
        os.rename(os.path.join(source_folder, file), os.path.join(destination_folder, file))

source_folder = os.path.join(wabad_training_folder, "Certhia familiaris_Eurasian Treecreeper")
dest_folder = "E:/Giacomo/Tovanella-20241110T120546Z-001/dataset_wabad_boosted/subset"
dest_folder = os.path.join(dest_folder, "Certhia familiaris_Eurasian Treecreeper").replace("\\", "/")
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
move_files_randomly(source_folder, dest_folder, 100)

## 2. audio_info_ext and category_info_ext

In [10]:
# import species_dict_map
with open('utils/species_dict_map.json') as json_file:
    species_dict_map = json.load(json_file)

In [12]:
def convert_to_json(input_file):
    audio_info = {}
    category_info = {}
    for txt_file in os.listdir(input_file):
        complete_path = os.path.join(input_file, txt_file)
        with open(complete_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile, delimiter='\t') 
            for row in reader:
                if row["Species"] not in all_species or "End Time (s)" not in row:
                    continue
                file_name = txt_file.replace(".txt", ".WAV")
                
                start_time = float(row["Begin Time (s)"])
                end_time = float(row["End Time (s)"])
                duration = end_time - start_time
                common_name = species_dict_map[row["Species"]]
                label = f"{row['Species']}_{common_name}"

                audio_info_entry = {
                    "scientific_name": row["Species"],
                    "common_name": common_name, 
                    "start_time": start_time,
                    "duration": duration,
                    "label": label
                }
                category_info_entry = {
                    "file_name": file_name,
                    "start_time": start_time,
                    "duration": duration,
                    "label": label
                }
                
                if file_name not in audio_info:
                    audio_info[file_name] = []
                if label not in category_info:
                    category_info[label] = []
                audio_info[file_name].append(audio_info_entry)
                category_info[label].append(category_info_entry)
    return audio_info, category_info

In [ ]:
wabad_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\WABAD"
audio_info_ext = {}
category_info_ext = {}
for folder in os.listdir(wabad_path):
    if not os.path.isdir(os.path.join(wabad_path, folder)):
        continue
    annotations = os.path.join(wabad_path, folder, folder, "Raven Pro annotations")
    audio_info_update, category_info_update = convert_to_json(annotations)
    for audio in audio_info_update.keys():
        if audio not in audio_info_ext:
            audio_info_ext[audio] = []
        audio_info_ext[audio].extend(audio_info_update[audio])
    for category in category_info_update.keys():
        if category not in category_info_ext:
            category_info_ext[category] = []
        category_info_ext[category].extend(category_info_update[category])    

# with open("utils/audio_info_ext.json", 'w', encoding='utf-8') as jsonfile:
#     json.dump(audio_info_ext, jsonfile)

# with open("utils/category_info_ext.json", 'w', encoding='utf-8') as jsonfile:
#     json.dump(category_info_ext, jsonfile)

## 3. refill

In [27]:
# se runnato, costruisce tutte le sessioni
true_segments = {}
target_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\ext_segments"
all_audios_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\WABAD\\"
species_list = category_info_ext.keys()
categories = list(species_list)
for j, category in enumerate(categories):
    all_category_audio = category_info_ext[category][:500]  # limit to 500 annotations
    total_length = len(all_category_audio)
    print(f"Loading {category} category... {j}/{len(categories)}")
    for i in tqdm(range(total_length)):
        audio_path = all_category_audio[i]["file_name"] # ex. BIAL_20210420_051500.WAV
        file_name, wav = audio_path.split('.')
        site = file_name.split('_')[0]
        start_time = all_category_audio[i]["start_time"]
        duration = all_category_audio[i]["duration"]
        start_times = []
        start_segms = []
        start_segm = int(start_time // 3)
        if duration < 1:    # ignore
            continue
        elif duration < 3:  # add contextual audio
            remaining_time = 3.0 - duration
            start_times.append(start_time - remaining_time / 2)
            start_segms.append(start_segm) 
        elif duration >= 3 and duration < 4:
            start_times.extend([start_time, start_time - 1.5, start_time + 1.5])
            start_segms.extend([start_segm, start_segm - 1, start_segm + 1]) 
        elif duration >= 4 and duration < 6:
            start_times.extend([start_time, start_time - 1.5, start_time + 3])
            start_segms.extend([start_segm, start_segm - 1, start_segm + 1]) 
        elif duration >= 6:
            start_times.extend([start_time - 1.5])
            start_segms.extend([start_segm - 1]) 
            num_full_segm = int(duration // 3)
            start_times.extend([start_time + i * 3.0 for i in range(num_full_segm)])
            start_segms.extend([start_segm + i for i in range(num_full_segm)]) 
            remaining_time = duration - 3.0 * num_full_segm
            if remaining_time >= 1:
                start_times.extend([start_time + 3.0 * num_full_segm])
                start_segms.extend([start_segm + num_full_segm]) 

        # print(start_times, start_segms)
        if audio_path not in true_segments:
            true_segments[audio_path] = {}
        for segm in start_segms:
            if segm not in true_segments[audio_path]:
                true_segments[audio_path][segm] = []
            true_segments[audio_path][segm].extend([category])
        # "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\WABAD\\BAM\\BAM\\Recordings"
        audio = AudioSegment.from_file(os.path.join(
            all_audios_path, site, site, "Recordings", audio_path), 
            format="wav"
        )
        for i, start_time in enumerate(start_times):
            export_path = os.path.join(
                target_path,
                category, 
                f"{file_name}_{start_segms[i]}.wav"
            )
            if os.path.exists(export_path):
                continue
            segment = audio[start_time*1000:start_time*1000 + 3000]
            os.makedirs(os.path.join(target_path, category), exist_ok=True)
            segment.export(export_path, format="wav")

Loading Phylloscopus collybita_Common Chiffchaff category... 0/22


  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:26<00:00, 18.74it/s]


Loading Phylloscopus trochilus_Willow Warbler category... 1/22


100%|██████████| 418/418 [00:12<00:00, 34.82it/s]


...

100%|██████████| 229/229 [00:06<00:00, 34.25it/s]


Loading Loxia curvirostra_Common Crossbill category... 20/22


100%|██████████| 25/25 [00:01<00:00, 15.15it/s]


Loading Muscicapa striata_Spotted Flycatcher category... 21/22


100%|██████████| 117/117 [00:04<00:00, 24.31it/s]


In [17]:
with open("utils/true_segments_ext.json", "w") as f:
    json.dump(true_segments, f)

## data augmentation

In [11]:
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    return audio, sr

def save_audio(file_path, audio, sr):
    sf.write(file_path, audio, sr)

In [12]:
from audiomentations import Compose, PitchShift, TimeStretch, AddBackgroundNoise

bg_noise = "e:\\Giacomo\\Tovanella-20241110T120546Z-001\\soundscapes\\soundscape_4.wav"
augmentations = {
    "ps": Compose([PitchShift(min_semitones=-1, max_semitones=1, p=1.0)]),
    "ts": Compose([TimeStretch(min_rate=0.95, max_rate=1.05, p=1.0)]),
    "bn": Compose([AddBackgroundNoise(sounds_path=bg_noise, p=1.0)]),
}

def apply_augmentations(file_path, output_dir):
    audio, sr = load_audio(file_path)
    
    for aug_name, aug in augmentations.items():
        augmented_audio = aug(samples=audio, sample_rate=sr)
        file_name = os.path.splitext(file_path)[0].split('\\')[-1]
        save_audio(f"{output_dir}/{file_name}_{aug_name}.wav", augmented_audio, sr)

In [ ]:
folder_path = "E:\\Giacomo\\Tovanella-20241110T120546Z-001\\dataset_wabad_boosted\\train\\Turdus viscivorus_Mistle Thrush"
# v1
# Poecile palustris_Marsh Tit
# Pyrrhula pyrrhula_Eurasian Bullfinch
# Dryocopus martius_Black Woodpecker
# v2
# Certhia familiaris_Eurasian Treecreeper
# Turdus viscivorus_Mistle Thrush
files = os.listdir(folder_path)
for i in tqdm(range(len(files))):
    audio = files[i]
    apply_augmentations(os.path.join(folder_path, audio), folder_path)

  0%|          | 0/308 [00:00<?, ?it/s]c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\audiomentations\core\audio_loading_utils.py:37: UserWarning: e:\Giacomo\Tovanella-20241110T120546Z-001\soundscapes\soundscape_4.wav had to be resampled from 48000 Hz to 24000 Hz. This hurt execution time.
  warnings.warn(
 48%|████▊     | 148/308 [00:19<00:20,  7.93it/s]c:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\audiomentations\core\audio_loading_utils.py:37: UserWarning: e:\Giacomo\Tovanella-20241110T120546Z-001\soundscapes\soundscape_4.wav had to be resampled from 48000 Hz to 44100 Hz. This hurt execution time.
  warnings.warn(
100%|██████████| 308/308 [00:39<00:00,  7.88it/s]


In [54]:
species_folders = os.listdir(wabad_training_folder)
# list all elements in each folder in folders
species_count = { folder: {
    # "train_count": len(os.listdir(os.path.join(training_folder, folder))),
    "WABAD_train_count": len(os.listdir(os.path.join(wabad_training_folder, folder))),
    "test_count": len(os.listdir(os.path.join(test_folder, folder)))
    } for folder in species_folders }
pd.DataFrame(species_count).T

,WABAD_train_count,test_count
Certhia familiaris_Eurasian Treecreeper,88,21
Dendrocopos major_Great Spotted Woodpecker,150,9
Dryocopus martius_Black Woodpecker,276,10
Erithacus rubecula_European Robin,1282,23
Fringilla coelebs_Common Chaffinch,3388,33
Lophophanes cristatus_Crested Tit,144,5
Loxia curvirostra_Common Crossbill,129,24
Muscicapa striata_Spotted Flycatcher,137,1
Pecking_,87,4
Periparus ater_Coal Tit,1448,41
